In [27]:
from typing import Tuple
import numpy as np


def sine(amplitude: float, frequency: float, time: float, phase: float) -> np.ndarray:
    return amplitude * np.sin(2 * np.pi * frequency * time + phase)


def tone(sample_rate: float, time_of_view: float, **kwargs: float) -> Tuple[np.ndarray, ...]:
    sample_period = 1 / sample_rate
    total_samples = time_of_view / sample_period
    time_points = np.linspace(0, time_of_view, int(total_samples))
    return time_points, sine(**kwargs, time=time_points)


def play_tone(signal: np.ndarray, play: bool) -> None:
    if play:
        sound_wave = signal.astype(np.int16)
        sd.stop()
        sd.play(sound_wave, blocking=False)
    else:
        sd.stop()


In [28]:
import sounddevice as sd

# TODO: find the proper frequencies for each tone
# Configure tone values
sampling_rate = 44_100
amplitude = 10_000
time_of_view = 2
phase = 0

# Configure the player
sd.default.samplerate = sampling_rate

# Map tone values to frequencies
tone_frequencies = [
    ('Do', 440),
    ('Re', 493.883),
    ('Mi', 554.3653),
    ('Fa', 587.3295),
    ('Sol', 659.2551),
    ('La', 739.9888),
    ('Si', 830.6094),
    ('Do\'', 880)
]

# Create tone values
time_points = None
tones = dict()

# Instantiate each tone
for (note, freq) in tone_frequencies:
    time_points, tones[note] = tone(
        amplitude=amplitude,
        frequency=freq,
        sample_rate=sampling_rate,
        time_of_view=time_of_view,
        phase=phase
    )


In [29]:
def play_sound(note, play):
    play_tone(tones[note], play)

In [30]:
from IPython.display import display
import ipywidgets as widgets

play_toggle = widgets.ToggleButton(
    value=False,
    description='Play Tone',
    disabled=False,
    button_style='',
    tooltip='Play Tone',
    icon='play'
)

dropdown = widgets.Dropdown(
    options=[note for (note, freq) in tone_frequencies],
    value=tone_frequencies[0][0],
    description='Tone'
)

controls = widgets.interactive(play_sound, note=dropdown, play=play_toggle)

h_box = widgets.HBox(
    controls.children
)

display(h_box)

In [19]:
import pretty_midi

# Transform MIDI note to frequency
# https://www.inspiredacoustics.com/en/MIDI_note_numbers_and_center_frequencies
def midi_note_to_freq(midi_note: int) -> float:
    if midi_note < 0 or midi_note > 127:
        raise Exception('A MIDI note must be between [0, 127]')
    return np.round(440 * np.power(2, (midi_note - 69) / 12), 2)

def create_song_from_midi(filepath, sample_rate, amplitude):
    # Read and parse the input midi file
    midi_data = pretty_midi.PrettyMIDI(filepath)

    # Configure the sound properties
    end_of_time = 0
    sampling_rate = sample_rate
    phase = 0

    # Configure the player
    sd.default.samplerate = sampling_rate

    # Find end of songs
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            end_of_time = max(end_of_time, note.end)

    # Create final composed song signal
    song_signal = np.zeros(int(np.ceil(end_of_time * sampling_rate)))

    # Create sound tones
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            # Calculate sound properties
            duration = note.end - note.start
            frequency = midi_note_to_freq(note.pitch)

            # Generate sinusoidal sound tone
            _, sound_note = tone(
                amplitude=amplitude,
                frequency=frequency,
                sample_rate=sampling_rate,
                time_of_view=duration,
                phase=phase
            )

            # Calculate when the tone enters the song
            start_of_tone = int(np.floor(sampling_rate * note.start))
            tone_duration = int(np.floor(sampling_rate * duration))

            # Compose song with the sound_tone at the proper position
            song_signal[start_of_tone:(start_of_tone + tone_duration)] += sound_note

    # Return the composed song singals
    return song_signal

In [32]:
import os

dir = './music-sheets'

midis = [
    f'{dir}/{filename}'
    for filename in os.listdir(dir)
]

amplitude_slider = widgets.FloatSlider(
    value=10_000,
    min=0,
    max=20_000,
    step=10,
    description='Amplitude:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

sample_rate_slider = widgets.FloatSlider(
    value=44_100,
    min=1_000,
    max=80_000,
    step=10,
    description='Sample Rate:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

play_toggle = widgets.ToggleButton(
    description='Play Song',
    disabled=False,
    button_style='',
    tooltip='Play Song',
    icon='play'
)

dropdown = widgets.Dropdown(
    options=midis,
    value='./music-sheets/ttls.mid',
    description='MIDI File:'
)

def play_sound(filepath, play_state, amplitude, sample_rate):
    if play_state == True:
        song = create_song_from_midi(filepath, sample_rate, amplitude)
    else:
        song = np.empty(1)

    play_tone(song, play_state)

widgets.interactive(
    play_sound,
    filepath=dropdown,
    play_state=play_toggle,
    amplitude=amplitude_slider,
    sample_rate=sample_rate_slider
)

display(
    widgets.HBox(
        (
            dropdown,
            play_toggle,
            amplitude_slider,
            sample_rate_slider,
        )
    ),
)